In [ ]:
##Instalar pacote de requisições SOAP
import sys
!{sys.executable} -m pip install zeep

In [ ]:
from zeep import Client
from zeep.wsse.username import UsernameToken

In [ ]:
client = Client('https://servicos.saude.gov.br/cnes/EstabelecimentoSaudeService/v1r0?wsdl', wsse=UsernameToken('CNES.PUBLICO', 'cnes#2015public'))

In [ ]:
client

In [ ]:
python -mzeep https://servicos.saude.gov.br/cnes/EstabelecimentoSaudeService/v1r0?wsdl

In [ ]:
zeep https://servicos.saude.gov.br/cnes/EstabelecimentoSaudeService/v1r0?wsdl

In [ ]:
%run python -mzeep https://servicos.saude.gov.br/cnes/EstabelecimentoSaudeService/v1r0?wsdl

In [ ]:
##Mostrar as informações do WSDL
client.wsdl.dump()

In [ ]:
request_data = {
    'FiltroPesquisaEstabelecimentoSaude': {
        'CodigoCNES': {
            'codigo' : 2530031
        }
    },
}

In [33]:
response = client.service.consultarEstabelecimentoSaude(**request_data)

In [34]:
print (response)

{
    'CodigoCNES': {
        'codigo': '2530031'
    },
    'CodigoUnidade': {
        'codigo': '2102002530031'
    },
    'nomeFantasia': {
        'Nome': 'HOSPITAL ADROALDO ALVES MATOS'
    },
    'nomeEmpresarial': {
        'Nome': 'PREFEITURA MUNICIPAL DE BOM JARDIM'
    },
    'CNPJ': None,
    'Endereco': {
        'identificador': None,
        'TipoEndereco': None,
        'TipoLogradouro': None,
        'nomeLogradouro': 'AV JOSE PEDRO VASCONCELOS',
        'numero': 'S/N',
        'complemento': 'BR 316',
        'Bairro': {
            'codigoBairro': None,
            'descricaoBairro': 'CENTRO'
        },
        'CEP': {
            'numeroCEP': '65380000'
        },
        'Municipio': {
            'codigoMunicipio': '210200',
            'nomeMunicipio': 'BOM JARDIM',
            'UF': {
                'codigoUF': '21',
                'siglaUF': 'MA',
                'codigoRegiao': None,
                'nomeUF': None
            }
        },
        'Pais': No